# Authors: Andrea ZANON, Hamza ZERHOUNI

In [1]:
using DataFrames
using JuMP, Gurobi
using DataFrames

Create first basic model

In [25]:
# number of teams
N = 4
N_CL = 1
# number of weeks
W = N-1
# number of days per weeks
D = 7; # we suppose week starts on tuesday, beacause monday is actually part of the previous football week

# define mondays, tuesdays, wednesdays, thursdays, fridays, saturdays, sundays. Suppose we start playing on friday
#=
fr = [7*k + 1 for k in 0:18]
sa = [7*k + 2 for k in 0:18]
su = [7*k + 3 for k in 0:18]
mo = [7*k + 4 for k in 0:18]
tu = [7*k + 5 for k in 0:18]
we = [7*k + 6 for k in 0:18]
th = [7*k + 7 for k in 0:18]
;
=#

# list with elements of fr, sa, su, mo


We define the matrix $C \in R^{TxT}$, with $C_{ij} = \frac{1}{j-i}$

In [22]:
C = zeros((D*(W+1), D*(W+1)))

for t in 1:W*(D)
    for t_ in t+1:W*(D)
        C[t,t_] = 1/sqrt(t_ - t)
    end
end

In [23]:
C

28×28 Matrix{Float64}:
 0.0  1.0  0.707107  0.57735   0.5       …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0       0.707107  0.57735      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0       1.0       0.707107     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0       0.0       1.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0       0.0       0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0       0.0       0.0       …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0       0.0       0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0       0.0       0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0       0.0       0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0       0.0       0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0       0.0       0.0       …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0       0.0       0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0       0.0       0.0       

Simple model, we are considering single league and making assumption that each team pla

In [5]:
model = Model(Gurobi.Optimizer)

# define variables
# 1 if team i plays team j on day d of week w, 0 otherwise
@variable(model, x[i = 1:N, j = 1:N, w = 0:W-1, d = 1:D], Bin)

# define objective
# remember week begins on tuesday, this is why w_ goes from w+1
@objective(model, Min, sum(C[7*w + d, 7 * (w+1) + d_]* x[i, j, w, d] * x[i, k, w+1, d_] for i in 1:N, j = 1:N, k = 1:N, w = 0:W-2, d = 1:D, d_ = 1:D))

# define constraints
# each team can never play itself
#@constraint(model, [i = 1:N, w = 0:W-1, d = 1:D], sum(x[i, i, w, d]) == 0)
@constraint(model, [i = 1:N, w = 0:W-1, d = 1:D], x[i, i, w, d] == 0)

# each team plays at most one team per week
@constraint(model, [i = 1:N, w = 0:W-1], sum(x[i, j, w, d] for j = 1:N, d = 1:D) <= 1)

# if team A plays team B, then team B plays team A
@constraint(model, [i = 1:N, j = 1:N, w = 0:W-1, d = 1:D], x[i, j, w, d] == x[j, i, w, d])

# each team plays exactly N-1 games
@constraint(model, [i = 1:N], sum(x[i, j, w, d] for j = 1:N, w = 0:W-1, d = 1:D) == N-1) # this constraint works

# all teams need to play each other exactly once
# @constraint(model, [i = 1:N, j = 1:N], sum(x[i, j, w, d] for w = 0:W-1, d = 1:D) == 1) # this makes it infeasible
@constraint(model, [i = 1:N, j = 1:N], sum(x[i, j, w, d] for w = 0:W-1, d = 1:D) <= 1) # this makes it infeasible


# at least four days of rest (if they play monday, they can't play again until saturday)
#@constraint(model, [i = 1:N, w = 0:W-2], sum(x[i, j, w+1, d1] for j = 1:N, d1 = 1:D) - sum(x[i, k, w, d2] for k = 1:N, d2 = 1:D) >= 4) # this makes it infeasible
@constraint(model, [i = 1:N, w = 0:W-2], sum((7*(w+1)+d1)*x[i, j, w+1, d1] for j = 1:N, d1 = 1:D) - sum((7*w+d2)x[i, k, w, d2] for k = 1:N, d2 = 1:D) >= 4) # this makes it infeasible


# can't play on tuesday (day 1), wednesday (day 2), thursday (day 3)
@constraint(model, [i = 1:N, j = 1:N, w = 0:W-1], sum(x[i, j, w, d] for d = 1:3) == 0)

# solve model
optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-12
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 508 rows, 336 columns and 2188 nonzeros
Model fingerprint: 0xd544c4d6
Model has 6272 quadratic objective terms
Variable types: 0 continuous, 336 integer (336 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [6e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 490 rows and 264 columns
Presolve time: 0.05s
Presolved: 42 rows, 96 columns, 1224 nonzeros
Variable types: 24 continuous, 72 integer (72 binary)
Found heuristic solution: objective 2.7475469

Root relaxation: cutoff, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Inc

In [6]:
x=value.(x)

4-dimensional DenseAxisArray{Float64,4,...} with index sets:
    Dimension 1, Base.OneTo(4)
    Dimension 2, Base.OneTo(4)
    Dimension 3, 0:2
    Dimension 4, Base.OneTo(7)
And data, a 4×4×3×7 Array{Float64, 4}:
[:, :, 0, 1] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 1, 1] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 2, 1] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 0, 2] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 1, 2] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 2, 2] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 0, 3] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 1, 3] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 2, 3] =
 0.

In [7]:
objective_value(model)

2.747546895706428

In [8]:
function calendar_week(x,week)
    df = DataFrame(Player=1:N, Monday=0.0, Tuesday=0.0, Wednesday=0.0, Thursday=0.0, Friday=0.0, Saturday=0.0, Sunday=0.0)
    for i=1:N
        for d=1:D
            for j=1:N
                if x[i,j,week,d] == 1.0
                    df[i,d+1] = j
                end
            end
        end
    end
    return df
end

calendar_week (generic function with 1 method)

In [9]:
### Week 1
calendar_week(0)

,Player,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,3.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,4.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [10]:
### Week 2
calendar_week(1)

,Player,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,4.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,3.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
### Week 3
calendar_week(2)

,Player,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,4.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [14]:
### Week 4
#calendar_week(3)

In [15]:
### Week 5
#calendar_week(4)

### Adding the CL to the model

In [68]:
model_CL = Model(Gurobi.Optimizer)

# define variables
# 1 if team i plays team j on day d of week w, 0 otherwise
@variable(model_CL, x[i = 1:N, j = 1:N, w = 0:W-1, d = 1:D], Bin)
@variable(model_CL, y[i = 1:N_CL, c = 1:N-1, w = 0:W-1, d = 1:D], Bin)

# define objective
# remember week begins on tuesday, this is why w_ goes from w+1
@objective(model_CL, Min, sum(C[d, d_]* y[i, c, w, d] * x[i, j, w, d_] for i in 1:N_CL, j = 1:N, c = 1:N-1, w = 0:W-1, d = 1:D, d_ = 1:D) 
    + sum(C[7*w + d, 7 * (w+1) + d_]* x[i, j, w, d] * y[i, c, w+1, d_] for i in 1:N_CL, j = 1:N, c = 1:N-1, w = 0:W-2, d = 1:D, d_ = 1:D)
    + sum(C[7*w + d, 7 * (w+1) + d_]* x[i, j, w, d] * x[i, k, w+1, d_] for i in 1:N, j = 1:N, k = 1:N, w = 0:W-2, d = 1:D, d_ = 1:D))

# define constraints
# each team can never play itself
#@constraint(model, [i = 1:N, w = 0:W-1, d = 1:D], sum(x[i, i, w, d]) == 0)
@constraint(model_CL, [i = 1:N, w = 0:W-1, d = 1:D], x[i, i, w, d] == 0)
# each team plays at most one team per week
@constraint(model_CL, [i = 1:N, w = 0:W-1], sum(x[i, j, w, d] for j = 1:N, d = 1:D) <= 1)
# if team A plays team B, then team B plays team A
@constraint(model_CL, [i = 1:N, j = 1:N, w = 0:W-1, d = 1:D], x[i, j, w, d] == x[j, i, w, d])
# each team plays exactly N-1 games
@constraint(model_CL, [i = 1:N], sum(x[i, j, w, d] for j = 1:N, w = 0:W-1, d = 1:D) == N-1) # this constraint works
# all teams need to play each other exactly once
@constraint(model_CL, [i = 1:N, j = 1:N], sum(x[i, j, w, d] for w = 0:W-1, d = 1:D) <= 1) # this makes it infeasible
# at least four days of rest (if they play monday, they can't play again until saturday)
@constraint(model_CL, [i = 1:N, w = 0:W-2], sum((7*(w+1)+d1)*x[i, j, w+1, d1] for j = 1:N, d1 = 1:D) - sum((7*w+d2)x[i, k, w, d2] for k = 1:N, d2 = 1:D) >= 4) # this makes it infeasible
# can't play on tuesday (day 1), wednesday (day 2), thursday (day 3)
@constraint(model_CL, [i = 1:N, j = 1:N, w = 0:W-1], sum(x[i, j, w, d] for d = 1:3) == 0)


@constraint(model_CL, [i = 1:N_CL, w = 0:W-1], sum(y[i, c, w, 1] + y[i, c, w, 2] for c=1:N-1) <= 1)
@constraint(model_CL, [i = 1:N_CL, w = 0:W-1], sum(y[i, c, w, d] for c=1:N-1, d=3:7) == 0)
@constraint(model_CL, [i = 1:N_CL], sum(y[i, c, w, d] for c=1:N-1, w=0:W-1, d=1:D) == N-1)
@constraint(model_CL, [i = 1:N_CL, c = 1:N-1], sum(y[i, c, w, d] for w=0:W-1, d=1:D) <= 1)


@constraint(model_CL, [i = 1:N_CL, w = 0:W-2], sum(d1*x[i, j, w, d1] for j=1:N, d1=1:D) - sum(d2*y[i, c, w, d2] for c=1:N-1, d2=1:D) >= 1)
@constraint(model_CL, [i = 1:N_CL, w = 0:W-2], sum((7*(w+1)+d2)*y[i, c, w+1, d2] for c=1:N-1, d2=1:D) - sum((7*(w)+d1)*x[i, j, w, d1] for j=1:N, d1=1:D) >= 1)



# solve model
optimize!(model_CL)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-12
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 522 rows, 399 columns and 2573 nonzeros
Model fingerprint: 0xb4b0e268
Model has 8204 quadratic objective terms
Variable types: 0 continuous, 399 integer (399 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [6e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 498 rows and 309 columns
Presolve time: 0.00s
Presolved: 72 rows, 138 columns, 1668 nonzeros
Variable types: 48 continuous, 90 integer (90 binary)
Found heuristic solution: objective 5.6567947

Root relaxation: objective 6.606302e-02, 67 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  

In [69]:
y = value.(y)
x = value.(x)

4-dimensional DenseAxisArray{Float64,4,...} with index sets:
    Dimension 1, Base.OneTo(4)
    Dimension 2, Base.OneTo(4)
    Dimension 3, 0:2
    Dimension 4, Base.OneTo(7)
And data, a 4×4×3×7 Array{Float64, 4}:
[:, :, 0, 1] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 1, 1] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 2, 1] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 0, 2] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 1, 2] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 2, 2] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 0, 3] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 1, 3] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 2, 3] =
 0.

In [70]:
function calendar_week_yCL(y,week)
    df = DataFrame(Player=1:N_CL, Monday=0.0, Tuesday=0.0, Wednesday=0.0, Thursday=0.0, Friday=0.0, Saturday=0.0, Sunday=0.0)
    for i=1:N_CL
        for d=1:D
            for c=1:N-1
                if y[i,c,week,d] == 1.0
                    df[i,d+1] = c
                end
            end
        end
    end
    return df
end

calendar_week_yCL (generic function with 1 method)

In [71]:
function calendar_week_xCL(x,week)
    df = DataFrame(Player=1:N, Monday=0.0, Tuesday=0.0, Wednesday=0.0, Thursday=0.0, Friday=0.0, Saturday=0.0, Sunday=0.0)
    for i=1:N
        for d=1:D
            for j=1:N
                if x[i,j,week,d] == 1.0
                    df[i,d+1] = j
                end
            end
        end
    end
    return df
end

calendar_week_xCL (generic function with 1 method)

In [72]:
calendar_week_CL(y,2)

,Player,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,3.0,0.0,0.0,0.0,0.0,0.0


In [73]:
calendar_week_xCL(x,2)

,Player,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0
